ITERATIVE FUNCTION

#We have the NU as the input, and it returns Y as the output. So we want to iterate that but changing the NU

In [1]:
import numpy as np
import copy
from itertools import product
import scipy as sp
import pylab as plt

from scipy.stats import gamma

from DMF_model import DMF_sim, DMF_parameters


def HRF(times):
    """ Return values for HRF at given times """
    # Gamma pdf for the peak
    peak_values = gamma.pdf(times, 6)
    # Gamma pdf for the undershoot
    undershoot_values = gamma.pdf(times, 12)
    # Combine them
    values = peak_values - 0.35 * undershoot_values
    # Scale max to 0.6
    return values / np.max(values) * 0.6    
    
    # DMF parameters
P = {}
P = DMF_parameters(P)
P['sigma'] = 0.02  # no noise

T  = 30     # simulation time
dt = 1e-4   # integration step
P['T'] = T

# layer specific external input
stim_start = int(10/dt)    # start of stimulation
stim_end   = int(20/dt)    # end of stimulation

# Define the possible values for each input intensity.
valuesI = [50, 100, 150, 200, 250, 300]
valuesLayer = range(8)

# Create an array of all possible combinations for each input
input_combinations = list(product(valuesI, repeat=2))
layer_combinations = list(product(valuesLayer, repeat=2))

# Initialize arrays to store the results
Y_results = []

for i in range(len(input_combinations)):
    # Update the U array based on the current population_combinations
    U = np.zeros((int(T/dt), P['M']))
    current_input_combination = input_combinations[i]    
    for j in range(len(layer_combinations)):
        current_layer_combination = layer_combinations[j]
        U[stim_start:stim_end, current_layer_combination[0]] = current_input_combination[0]
        U[stim_start:stim_end, current_layer_combination[1]] = current_input_combination[1]
         # Simulate
        I, H, F = DMF_sim(U, P)     # I - input current, H - membrane potential, F - firing rate
        Y = I[int(1/dt):, :] - np.mean(I[int(0.5/dt):int(1/dt), :], axis=0)  # deviation from baseline during stimulation (and remove initial transient)
        Y = (Y - np.min(Y)) / (np.max(Y) - np.min(Y)) * 2 - 1  # normalize Y between -1 and 1

        # Append the Y values to the results
        Y_results.append(Y)

Y_results_array = np.array(Y_results)

KeyboardInterrupt: 

In [26]:
a = range(len(input_combinations))
print(a)

range(0, 36)


In [42]:
from itertools import product

valuesI = [50, 100, 150, 200, 250, 300]
input_combinations = list(product(valuesI, repeat=2))
a = list(input_combinations)
b = len(a)
c = input_combinations[3]
U = [0]*40
for i in range(len(input_combinations)):
    U[i] = input_combinations[i]
    
print(a[3])

SyntaxError: invalid syntax (1341353130.py, line 12)

In [50]:
Y_results_array.shape


(81, 490000, 8)

In [51]:
print(U)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Current: NU, more combinations. input ranges from 0 to 300 in incrememnts of possibly 25mA? add noise with the same combinations, sample around the value of th einput (randomly sampled as a normal distribution), 

Alex's notes:
- The recording matrix needs three dimensions. Condition, Layer and Timestep of the simulation. 
- Start from 0 firing rate to 300. The intensity should be from 50 to 300 picoA.

- Simulating each condition severlat times.
- Randomize the input intensity around a normal distribution for each point. 
	- So if the algorithm says that the input for the function is 100, instead it is:
	- I = 100 +- Normal Dist around 100. The error is for us to decide. Think of it ourselves. Depends on what the length of the steps are. 

Ang's notes:
- reduce the stimulation time (10 s) porribly start at 10 finish at 15? or 12?
- add noise to the input by creating random value dx (+-(0 to 10?), save that in current_input
- new noise every iteration? or every time step?
